In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from scipy.interpolate import griddata
from deap import base, creator, tools, algorithms
import random

In [ ]:
# Load data from the .mat file
data = sio.loadmat('../data/ML_Optimal_method.mat')
eddy_double_LES = data['eddy_double_LES']
eddy_double_TBNN = data['eddy_double_TBNN']
Sij = data['Sij']
Rij = data['Rij']
x = data['x']
y = data['y']
x_point = data['x_point']
y_point = data['y_point']
bij_LES = data['bij_LES']
bij_TBNN = data['bij_TBNN']

### edd viscosity = function(Sij, Rij)

In [ ]:
# Prepare features and target variables
# Use columns 0, 1, 3, and 4 from Sij an Rij as features
X = np.concatenate((Sij[:, [0, 1, 3, 4]], Rij[:, [0, 1, 3, 4]]), axis=1)
Y = eddy_double_TBNN.ravel()
Y_LES = eddy_double_LES.ravel()

# For simplicity, we do not scale the features here (you can scale if needed)

# Prepare independent variables as a tuple (each as a 1-D array) for all 8 features
X1 = X[:, 0]
X2 = X[:, 1]
X3 = X[:, 2]
X4 = X[:, 3]
X5 = X[:, 4]
X6 = X[:, 5]
X7 = X[:, 6]
X8 = X[:, 7]
X_tuple = (X1, X2, X3, X4, X5, X6, X7, X8)

# Define a candidate linear function using 8 features:
# y = a + b*x1 + c*x2 + d*x3 + e*x4 + f*x5 + g*x6 + h*x7 + i*x8
def candidate_func_linear(X_tuple, params):
    a, b, c, d, e, f, g, h, i = params
    x1, x2, x3, x4, x5, x6, x7, x8 = X_tuple
    return a + b * x1 + c * x2 + d * x3 + e * x4 + f * x5 + g * x6 + h * x7 + i * x8

# Define the objective function (MSE) to minimize
def evalCandidate(individual):
    params = individual
    Y_pred = candidate_func_linear(X_tuple, params)
    mse = mean_squared_error(Y, Y_pred)
    return (mse,)

# Set up DEAP for minimization
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimization
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
# Each parameter is a float in range [-10, 10]
toolbox.register("attr_float", random.uniform, -10, 10)
# An individual consists of 9 parameters (for the intercept and 8 coefficients)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=9)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register evaluation, crossover, mutation, and selection operators
toolbox.register("evaluate", evalCandidate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=2.0, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Set random seed for reproducibility
random.seed(42)

# Create initial population
population = toolbox.population(n=200)

# Define statistics and Hall-of-Fame to record the best individual
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
hof = tools.HallOfFame(1)

# Run the genetic algorithm for 50 generations
population, log = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=2000, 
                                      stats=stats, halloffame=hof, verbose=True)

best_params = hof[0]
print("Best linear parameters found by GA:", best_params)

# Compute predictions using the best individual from GA
Y_pred_ga = candidate_func_linear(X_tuple, best_params)
rmse_ga = np.sqrt(mean_squared_error(Y, Y_pred_ga))
r2_ga = r2_score(Y, Y_pred_ga)
print("GA Linear Model RMSE:", rmse_ga)
print("GA Linear Model R2:", r2_ga)

# Plot predictions vs. actual values for the GA-based linear model
plt.figure(figsize=(10,6))
plt.plot(Y, label="Actual eddy_double_TBNN", color='blue')
plt.plot(Y_pred_ga, label="GA Linear Prediction", color='red', linestyle='--')
plt.xlabel("Sample Index")
plt.ylabel("Value")
plt.title("GA-based Linear Model Prediction vs. Actual")
plt.legend()
plt.grid(True)
plt.show()

# Interpolate the results onto the spatial grid
points = np.column_stack((x_point.ravel(), y_point.ravel()))
eddy_double_LES_interp = griddata(points, eddy_double_LES.ravel(), (x, y), method='cubic').reshape(x.shape)
eddy_double_TBNN_interp = griddata(points, eddy_double_TBNN.ravel(), (x, y), method='cubic').reshape(x.shape)
Y_pred_interp = griddata(points, Y_pred_ga.ravel(), (x, y), method='cubic').reshape(x.shape)

# Set common color scale limits for the spatial plots
vmin = -0.05
vmax = 0.20

# Plot the interpolated maps in a vertical stack (3 subplots)
fig, axs = plt.subplots(3, 1, figsize=(8, 8))
im0 = axs[0].pcolormesh(x, y, eddy_double_LES_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[0].set_title('eddy double LES')
axs[0].set_aspect('equal', adjustable='box')
axs[0].set_xlim(np.min(x), np.max(x))
axs[0].set_ylim(np.min(y), np.max(y))

im1 = axs[1].pcolormesh(x, y, eddy_double_TBNN_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[1].set_title('eddy double TBNN')
axs[1].set_aspect('equal', adjustable='box')
axs[1].set_xlim(np.min(x), np.max(x))
axs[1].set_ylim(np.min(y), np.max(y))

im2 = axs[2].pcolormesh(x, y, Y_pred_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[2].set_title('GA-based Linear Y pred')
axs[2].set_aspect('equal', adjustable='box')
axs[2].set_xlim(np.min(x), np.max(x))
axs[2].set_ylim(np.min(y), np.max(y))

fig.subplots_adjust(right=0.85, hspace=0.1)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(im2, cax=cbar_ax)
plt.show()

### edd viscosity = function(Sij)

In [ ]:
# Prepare features and target variable
# Use columns 0, 1, 3, and 4 from Sij as features
X = Sij[:, [0, 1, 3, 4]]
Y = eddy_double_TBNN.ravel()
Y_LES = eddy_double_LES.ravel()

# Standardize features manually (if needed, here we skip for simplicity)
# In a linear model, scaling might still help, but here we assume raw values

# Prepare independent variables as a tuple (each as a 1-D array)
X1 = X[:, 0]
X2 = X[:, 1]
X3 = X[:, 2]
X4 = X[:, 3]
X_tuple = (X1, X2, X3, X4)

# Define a candidate linear function:
# y = a + b*x1 + c*x2 + d*x3 + e*x4
def candidate_func_linear(X_tuple, params):
    a, b, c, d, e = params
    x1, x2, x3, x4 = X_tuple
    return a + b * x1 + c * x2 + d * x3 + e * x4

# Define the objective function to minimize (MSE)
def evalCandidate(individual):
    params = individual
    Y_pred = candidate_func_linear(X_tuple, params)
    mse = mean_squared_error(Y, Y_pred)
    return (mse,)

# Set up DEAP for minimization
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimization
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
# Each parameter is a float in range [-10, 10]
toolbox.register("attr_float", random.uniform, -10, 10)
# An individual consists of 5 parameters
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=5)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register the evaluation function, crossover, mutation and selection operators
toolbox.register("evaluate", evalCandidate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=2.0, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Set random seed for reproducibility
random.seed(42)

# Create initial population
population = toolbox.population(n=200)

# Define statistics and hall-of-fame to record best individual
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
hof = tools.HallOfFame(1)

# Run the genetic algorithm for 50 generations
population, log = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=10000, 
                                      stats=stats, halloffame=hof, verbose=True)

best_params = hof[0]
print("Best linear parameters found by GA:", best_params)

# Compute predictions using the best individual from GA
Y_pred_ga = candidate_func_linear(X_tuple, best_params)
rmse_ga = np.sqrt(mean_squared_error(Y, Y_pred_ga))
r2_ga = r2_score(Y, Y_pred_ga)
print("GA Linear Model RMSE:", rmse_ga)
print("GA Linear Model R2:", r2_ga)

# Plot predictions vs. actual values for the GA-based linear model
plt.figure(figsize=(10,6))
plt.plot(Y, label="Actual eddy_double_TBNN", color='blue')
plt.plot(Y_pred_ga, label="GA Linear Prediction", color='red', linestyle='--')
plt.xlabel("Sample Index")
plt.ylabel("Value")
plt.title("GA-based Linear Model Prediction vs. Actual")
plt.legend()
plt.grid(True)
plt.show()

# Interpolate the results onto the spatial grid
points = np.column_stack((x_point.ravel(), y_point.ravel()))
eddy_double_LES_interp = griddata(points, eddy_double_LES.ravel(), (x, y), method='cubic').reshape(x.shape)
eddy_double_TBNN_interp = griddata(points, eddy_double_TBNN.ravel(), (x, y), method='cubic').reshape(x.shape)
Y_pred_interp = griddata(points, Y_pred_ga.ravel(), (x, y), method='cubic').reshape(x.shape)

# Set common color scale limits for the spatial plots
vmin = -0.05
vmax = 0.20

# Plot the interpolated maps in a vertical stack (3 subplots)
fig, axs = plt.subplots(3, 1, figsize=(8, 8))
im0 = axs[0].pcolormesh(x, y, eddy_double_LES_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[0].set_title('eddy double LES')
axs[0].set_aspect('equal', adjustable='box')
axs[0].set_xlim(np.min(x), np.max(x))
axs[0].set_ylim(np.min(y), np.max(y))

im1 = axs[1].pcolormesh(x, y, eddy_double_TBNN_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[1].set_title('eddy double TBNN')
axs[1].set_aspect('equal', adjustable='box')
axs[1].set_xlim(np.min(x), np.max(x))
axs[1].set_ylim(np.min(y), np.max(y))

im2 = axs[2].pcolormesh(x, y, Y_pred_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[2].set_title('GA-based Linear Y pred')
axs[2].set_aspect('equal', adjustable='box')
axs[2].set_xlim(np.min(x), np.max(x))
axs[2].set_ylim(np.min(y), np.max(y))

fig.subplots_adjust(right=0.85, hspace=0.1)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(im2, cax=cbar_ax)
plt.show()


### edd viscosity = function(Sij,bij)

In [ ]:
# Prepare features and target variables
# Use columns 0, 1, 3, and 4 from Sij and bij_TBNN as features
X = np.concatenate((Sij[:, [0, 1, 3, 4]], bij_TBNN[:, [0, 1, 3, 4]]), axis=1)
Y = eddy_double_TBNN.ravel()
Y_LES = eddy_double_LES.ravel()

# For simplicity, we do not scale the features here (you can scale if needed)

# Prepare independent variables as a tuple (each as a 1-D array) for all 8 features
X1 = X[:, 0]
X2 = X[:, 1]
X3 = X[:, 2]
X4 = X[:, 3]
X5 = X[:, 4]
X6 = X[:, 5]
X7 = X[:, 6]
X8 = X[:, 7]
X_tuple = (X1, X2, X3, X4, X5, X6, X7, X8)

# Define a candidate linear function using 8 features:
# y = a + b*x1 + c*x2 + d*x3 + e*x4 + f*x5 + g*x6 + h*x7 + i*x8
def candidate_func_linear(X_tuple, params):
    a, b, c, d, e, f, g, h, i = params
    x1, x2, x3, x4, x5, x6, x7, x8 = X_tuple
    return a + b * x1 + c * x2 + d * x3 + e * x4 + f * x5 + g * x6 + h * x7 + i * x8

# Define the objective function (MSE) to minimize
def evalCandidate(individual):
    params = individual
    Y_pred = candidate_func_linear(X_tuple, params)
    mse = mean_squared_error(Y, Y_pred)
    return (mse,)

# Set up DEAP for minimization
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimization
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
# Each parameter is a float in range [-10, 10]
toolbox.register("attr_float", random.uniform, -10, 10)
# An individual consists of 9 parameters (for the intercept and 8 coefficients)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=9)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register evaluation, crossover, mutation, and selection operators
toolbox.register("evaluate", evalCandidate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=2.0, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Set random seed for reproducibility
random.seed(42)

# Create initial population
population = toolbox.population(n=200)

# Define statistics and Hall-of-Fame to record the best individual
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
hof = tools.HallOfFame(1)

# Run the genetic algorithm for 50 generations
population, log = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=2000, 
                                      stats=stats, halloffame=hof, verbose=True)

best_params = hof[0]
print("Best linear parameters found by GA:", best_params)

# Compute predictions using the best individual from GA
Y_pred_ga = candidate_func_linear(X_tuple, best_params)
rmse_ga = np.sqrt(mean_squared_error(Y, Y_pred_ga))
r2_ga = r2_score(Y, Y_pred_ga)
print("GA Linear Model RMSE:", rmse_ga)
print("GA Linear Model R2:", r2_ga)

# Plot predictions vs. actual values for the GA-based linear model
plt.figure(figsize=(10,6))
plt.plot(Y, label="Actual eddy_double_TBNN", color='blue')
plt.plot(Y_pred_ga, label="GA Linear Prediction", color='red', linestyle='--')
plt.xlabel("Sample Index")
plt.ylabel("Value")
plt.title("GA-based Linear Model Prediction vs. Actual")
plt.legend()
plt.grid(True)
plt.show()

# Interpolate the results onto the spatial grid
points = np.column_stack((x_point.ravel(), y_point.ravel()))
eddy_double_LES_interp = griddata(points, eddy_double_LES.ravel(), (x, y), method='cubic').reshape(x.shape)
eddy_double_TBNN_interp = griddata(points, eddy_double_TBNN.ravel(), (x, y), method='cubic').reshape(x.shape)
Y_pred_interp = griddata(points, Y_pred_ga.ravel(), (x, y), method='cubic').reshape(x.shape)

# Set common color scale limits for the spatial plots
vmin = -0.05
vmax = 0.20

# Plot the interpolated maps in a vertical stack (3 subplots)
fig, axs = plt.subplots(3, 1, figsize=(8, 8))
im0 = axs[0].pcolormesh(x, y, eddy_double_LES_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[0].set_title('eddy double LES')
axs[0].set_aspect('equal', adjustable='box')
axs[0].set_xlim(np.min(x), np.max(x))
axs[0].set_ylim(np.min(y), np.max(y))

im1 = axs[1].pcolormesh(x, y, eddy_double_TBNN_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[1].set_title('eddy double TBNN')
axs[1].set_aspect('equal', adjustable='box')
axs[1].set_xlim(np.min(x), np.max(x))
axs[1].set_ylim(np.min(y), np.max(y))

im2 = axs[2].pcolormesh(x, y, Y_pred_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[2].set_title('GA-based Linear Y pred')
axs[2].set_aspect('equal', adjustable='box')
axs[2].set_xlim(np.min(x), np.max(x))
axs[2].set_ylim(np.min(y), np.max(y))

fig.subplots_adjust(right=0.85, hspace=0.1)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(im2, cax=cbar_ax)
plt.show()